In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import lightgbm as lgb
import xgboost as xgb
import joblib
import os

In [3]:
import xgboost
print(xgboost.__version__)


3.0.5


In [4]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
print(type(xgb_model))


<class 'xgboost.sklearn.XGBRegressor'>


In [5]:
# Load synthetic data
df = pd.read_csv("../data/sales_data.csv", parse_dates=['date'])


In [6]:
# Feature engineering
df['moving_avg_demand'] = df.groupby('product_id')['units_sold'].transform(lambda x: x.rolling(7, min_periods=1).mean())
df['price_elasticity'] = (df['units_sold'].pct_change() / df['historical_price'].pct_change()).fillna(0)
df['trend_factor'] = df.groupby('product_id')['units_sold'].transform(lambda x: x.rolling(30, min_periods=1).mean())

features = ['units_sold', 'competitor_price', 'stock_level', 'day_of_week', 'holiday_flag',
            'views', 'moving_avg_demand', 'price_elasticity', 'trend_factor']
target = 'historical_price'

In [7]:
X = df[features]
y = df[target]

In [8]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
import numpy as np
import pandas as pd

print(np.isinf(X_train).sum())   # Count of infinite values
print(np.isnan(X_train).sum())   # Count of NaN values


units_sold                0
competitor_price          0
stock_level               0
day_of_week               0
holiday_flag              0
views                     0
moving_avg_demand         0
price_elasticity     116584
trend_factor              0
dtype: int64
units_sold           0
competitor_price     0
stock_level          0
day_of_week          0
holiday_flag         0
views                0
moving_avg_demand    0
price_elasticity     0
trend_factor         0
dtype: int64


In [13]:
# Replace inf values with NaN first
X_train = X_train.replace([np.inf, -np.inf], np.nan)
X_test = X_test.replace([np.inf, -np.inf], np.nan)

# Fill NaN values
X_train['price_elasticity'] = X_train['price_elasticity'].fillna(0)
X_test['price_elasticity'] = X_test['price_elasticity'].fillna(0)

# Verify
print(X_train.isnull().sum())
print(X_test.isnull().sum())


units_sold           0
competitor_price     0
stock_level          0
day_of_week          0
holiday_flag         0
views                0
moving_avg_demand    0
price_elasticity     0
trend_factor         0
dtype: int64
units_sold           0
competitor_price     0
stock_level          0
day_of_week          0
holiday_flag         0
views                0
moving_avg_demand    0
price_elasticity     0
trend_factor         0
dtype: int64


In [14]:
lgb_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=7,
    random_state=42
)

# Use callbacks for early stopping and logging
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

joblib.dump(lgb_model, '../models/lgb_model.pkl')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1091
[LightGBM] [Info] Number of data points in the train set: 146400, number of used features: 9
[LightGBM] [Info] Start training from score 275.740827
Training until validation scores don't improve for 50 rounds
[50]	valid_0's l2: 194.668
[100]	valid_0's l2: 98.0308
[150]	valid_0's l2: 97.3593
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

['../models/lgb_model.pkl']

In [16]:
import xgboost as xgb
from xgboost.callback import EarlyStopping

# Convert to DMatrix (XGBoost’s native format)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

params = {
    "objective": "reg:squarederror",
    "learning_rate": 0.05,
    "max_depth": 7,
    "eval_metric": "mae"
}

# Train with early stopping
bst = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    evals=[(dtest, "validation")],
    callbacks=[EarlyStopping(rounds=10, save_best=True)]
)

print("Best iteration:", bst.best_iteration)
print("Best score:", bst.best_score)
joblib.dump(lgb_model, '../models/xgb_moxdel.pkl')


[0]	validation-mae:103.39464
[1]	validation-mae:98.25597
[2]	validation-mae:93.37557
[3]	validation-mae:88.74124
[4]	validation-mae:84.33767
[5]	validation-mae:80.15652
[6]	validation-mae:76.18469
[7]	validation-mae:72.41474
[8]	validation-mae:68.83322
[9]	validation-mae:65.43345
[10]	validation-mae:62.20584
[11]	validation-mae:59.14166
[12]	validation-mae:56.23220
[13]	validation-mae:53.47088
[14]	validation-mae:50.84958
[15]	validation-mae:48.36273
[16]	validation-mae:46.00198
[17]	validation-mae:43.76359
[18]	validation-mae:41.63969
[19]	validation-mae:39.62636
[20]	validation-mae:37.71585
[21]	validation-mae:35.90444
[22]	validation-mae:34.18773
[23]	validation-mae:32.56018
[24]	validation-mae:31.01853
[25]	validation-mae:29.55871
[26]	validation-mae:28.17604
[27]	validation-mae:26.86707
[28]	validation-mae:25.62770
[29]	validation-mae:24.45620
[30]	validation-mae:23.34763
[31]	validation-mae:22.30014
[32]	validation-mae:21.31081
[33]	validation-mae:20.37618
[34]	validation-mae:19.

['../models/xgb_moxdel.pkl']

In [19]:
def evaluate_model(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # manual RMSE
    print(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")
    return mae, rmse

# LightGBM predictions
y_pred_lgb = lgb_model.predict(X_test)
evaluate_model(y_test, y_pred_lgb, "LightGBM")

# XGBoost predictions
y_pred_xgb = bst.predict(dtest)
evaluate_model(y_test, y_pred_xgb, "XGBoost")

LightGBM - MAE: 8.1385, RMSE: 9.8669
XGBoost - MAE: 8.1256, RMSE: 9.8560


(8.125600814819336, np.float64(9.856024171205085))